# Multihash PCY Algorithm #

In [3]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import itertools as iter_t

retail_data = [i.strip().split() for i in open("retail.dat").readlines()]

retail_data = pd.DataFrame(retail_data)

# allocate a limited number of rows, for testing
retail_data = retail_data[:100]

In [4]:
def multihash_PCY(data, chunks=2 ,f_hash, g_hash, min_support=0.05, min_confidence=0.10):
    
    #candidate checking sub-method
    def n_candidate(prev_L, n, hash_f, hash_d = None):
        #generate possible sets
        names_helper = np.array([], dtype=int)
        for i in range(n-1):
            names_helper = np.append(names_helper, prev_L[i].to_numpy(dtype=int))
        
        #create 2x array of sets to store frequency
        candidate         = iter_t.combinations(set(names_helper), n)
        candidate         = pd.DataFrame([i for i in candidate])
        candidate["freq"] = np.zeros(len(candidate[0]), dtype=int)
        
        candidate.is_copy = False
        
        if hash_d is not None:
            #retrieve hash values
            for i in range(len(candidate[0])):
                p = candidate.iloc[i, 0:n]
                h = hash_f(p[0], p[1])
                v = hash_d.get(h)
                #remove non-frequent sets
                if v is None:
                    candidate.drop(candidate.index[i])
                elif v == 0:
                    candidate.drop(candidate.index[i])
        
        for i in range(data.shape[0]):
            for j in range(candidate.shape[0]):
                if set(candidate.loc[j][0:n].to_numpy(dtype=int)).issubset(set(data.loc[i].dropna().to_numpy(dtype=int))):
                    candidate.loc[j, "freq"] += 1
        return candidate
    
    
    # get unique items
    # construct array of all items
    items     = []
    hash_dict = {}
    hash_dict2 = {}
    #for each array item
    for i in range(data.shape[1]):
        for j in range(data.shape[0]):
            #copy data from the dataset
            items.append(data[i][j])
        pairs = iter_t.combinations(set(data[i]), 2)
        for p in pairs:
            if (p[0] is not None) and (p[1] is not None):
                #first hashmap
                h = f_hash(p[0], p[1])
                v = hash_dict.get(h)
                if v is None:
                    hash_dict[h] = 1
                else:
                    hash_dict[h] += 1
                #second hashmap
                w = hash_dict2.get(g)
                g = g_hash(p[0], p[1])
                if w is None:
                    hash_dict2[g] = 1
                else:
                    hash_dict2[g] += 1
    #check support of each set
    for k in hash_dict.keys():
        if hash_dict[k] >= min_support:
            hash_dict[k] = 1
        else:
            hash_dict[k] = 0
            
    for k in hash_dict2.keys():
        if hash_dict2[k] >= min_support:
            hash_dict2[k] = 1
        else:
            hash_dict2[k] = 0
        
    unique_items = list(set(items))
    
    # construct candidate sets
    C = []
    L = []
    
    C1 = []
    for val in unique_items:
        C1.append((val, items.count(val)))
    
    total_transactions = len(data)
    
    del(items)
    
    C1 = pd.DataFrame(C1, columns=[0, "freq"], dtype=int)
    C1 = C1.dropna()
    
    C1["conf"] = np.ones(len(C1[0]), dtype=float)
    C1["sup"]  = C1["freq"] / total_transactions
    
    #keep sets that match or exceed threshold
    L1 = C1[C1["sup"] >= min_support]
    L1 = L1.astype({0: int})
    
    C.append(C1)
    L.append(L1)
    
    def conf(_L, prevL, n):
        # Build confidence
        pd.set_option('mode.chained_assignment', 'warn')
        _L["conf"] = np.zeros(len(_L[0]), dtype=float)

        _L.is_copy = False
        for i in range(len(_L["freq"])):
            oldSup = _L["freq"].iloc[i]                                        #(prevL["freq"][prevL.iloc[:, 0:n] == L_cur[0:n]]).iat[0]
            for j in range(len(prevL[0])):
                if set(prevL.iloc[j, 0:n-1]) == set(_L.iloc[i, 0:n-1]):
                    oldSup = prevL["freq"].iloc[j]
                    break
            _L["conf"].iloc[i] = (_L["freq"].iloc[i] / oldSup)
        
        return _L
        
    i = 2
    while True:
        cand         = n_candidate(L[i-2], i, f_hash, hash_dict)
        hash_dict    = None
        cand["sup"]  = cand["freq"] / total_transactions
        L_cur        = cand[cand["sup"] >= min_support]
        
        cand2        = n_candidate(L[i-2], i, g_hash, hash_dict2)
        hash_dict2    = None
        cand2["sup"]  = cand2["freq"] / total_transactions
        L_cur2        = cand2[cand2["sup"] >= min_support]
        
        if len(L_cur[0]) == 0 or len(L_cur2[0]) == 0:
            break
        
        L_cur = conf(L_cur, L[i-2], i)
        L_cur2 = conf(L_cur2, L[i-2], i)
        
        L_cur = L_cur[L_cur["conf"] >= min_confidence]
        L_cur2 = L_cur2[L_cur2["conf"] >= min_confidence]
        
        #if the candidate is above min_condfidence in both bitmaps:
        if(L_cur is not None and L_cur2 is not None):
            L.append(L_cur)
            C.append(cand)
        
        i += 1
    
    return L

In [5]:
res = multihash_PCY(retail_data, chunks=2, lambda x, y: (int(x)*int(y))**3 % 300, lambda x, y: (int(3*x)*int(2*y))**4 % 1000)
res

/home/oboutte/.local/lib/python2.7/site-packages/pandas/core/generic.py:5079: FutureWarning: Attribute 'is_copy' is deprecated and will be removed in a future version.
  object.__getattribute__(self, name)
/home/oboutte/.local/lib/python2.7/site-packages/pandas/core/generic.py:5080: FutureWarning: Attribute 'is_copy' is deprecated and will be removed in a future version.
  return object.__setattr__(self, name, value)
/home/oboutte/.local/lib/python2.7/site-packages/ipykernel_launcher.py:83: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
/home/oboutte/.local/lib/python2.7/site-packages/pandas/core/indexing.py:190: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/

KeyError: 0